# Create Rhea reference files

1. REF_RHEA2MASTER: dictionary of rhea to master
2. REF_RHEA2LABEL: dictionary of rhea to label, i.e., the equations of the reactions
3. REF_RHEA2CHEBI: dictionary of rhea to chebi, i.e., the chebi terms that are the participants of the reaction
4. REF_MAT: reference matrix of rhea to chebi, i.e., the reference matrix of the reactions
5. REF_RHEA2ECKEGG, REF_EC2RHEA, REF_KEGG2RHEA: dictionaries of rhea to ec and kegg, and the reverse mappings

In [5]:
import compress_pickle
import itertools
import numpy as np
import os
import pandas as pd
import pickle
import requests
import io

DATA_DIR = '/Users/luna/Desktop/CRBM/AMAS_proj/Data'

## Data
### Direction data
Data are downloaded from the Rhea website: https://www.rhea-db.org/help/download.   
TSV file of the mapping of reaction directions (rhea-directions.tsv) is obtained using the link: https://ftp.expasy.org/databases/rhea/tsv/rhea-directions.tsv.

### Reaction information
Reaction information (identifier, chebi names, chebi ids) can be obtained from Rhea REST API (https://www.rhea-db.org/help/rest-api).

In [74]:
url= "https://www.rhea-db.org/rhea?"
parameter = {
  "query":'',
  "columns":"rhea-id,equation,chebi,chebi-id,ec,reaction-xref(KEGG)",
  "format":'tsv'
}
response = requests.get(url,params=parameter)
df = pd.read_csv(io.StringIO(response.text), delimiter='\t', keep_default_na=False)
print(df.shape)
df.head()

(17131, 6)


,Reaction identifier,Equation,ChEBI name,ChEBI identifier,EC number,Cross-reference (KEGG)
0,RHEA:22832,N-formimidoyl-L-glutamate + H2O = N-formyl-L-g...,N-formimidoyl-L-glutamate;H2O;N-formyl-L-gluta...,CHEBI:58928;CHEBI:15377;CHEBI:17684;CHEBI:28938,EC:3.5.3.13,KEGG:R02286
1,RHEA:22836,a fatty acyl-[ACP] + malonyl-[ACP] + H(+) = a ...,O-(S-fatty acylpantetheine-4'-phosphoryl)-L-se...,CHEBI:138651;CHEBI:78449;CHEBI:15378;CHEBI:787...,EC:2.3.1.41,KEGG:R02768
2,RHEA:22840,D-glutamine + H2O = D-glutamate + NH4(+),D-glutamine;H2O;D-glutamate;NH4(+),CHEBI:58000;CHEBI:15377;CHEBI:29986;CHEBI:28938,EC:3.5.1.35,KEGG:R01579
3,RHEA:22844,"(6aS,11aS)-4-dimethylallyl-3,6a,9-trihydroxypt...","(6aS,11aS)-4-dimethylallyl-3,6a,9-trihydroxypt...",CHEBI:50036;CHEBI:57618;CHEBI:15379;CHEBI:1647...,EC:1.14.14.135,KEGG:R07197
4,RHEA:22848,(3S)-hydroxyoctanedioyl-CoA + NAD(+) = 3-oxooc...,(3S)-hydroxyoctanedioyl-CoA;NAD(+);3-oxooctane...,CHEBI:76333;CHEBI:57540;CHEBI:76335;CHEBI:5794...,,


## REF_RHEA2MASTER

Because there are seperate ids for the same reaction in different directions, we need to map the reaction to the master direction.

In [26]:
df_directions = pd.read_csv(os.path.join(DATA_DIR, "rhea-directions_jan2025.tsv"), delimiter='\t', keep_default_na=False)
print(df_directions.shape)

(17131, 4)


In [ ]:
all2rhea_bi = dict()
all2rhea_master = dict()

for idx in df_directions.index:
  one_row = df_directions.loc[idx, :]
  # updating all2bi
  for direction_type in one_row.index:
    all2rhea_bi['RHEA:'+str(one_row[direction_type])] = 'RHEA:'+str(one_row['RHEA_ID_BI'])
    all2rhea_master['RHEA:'+str(one_row[direction_type])] = 'RHEA:'+str(one_row['RHEA_ID_MASTER'])

compress_pickle.dump(all2rhea_master, os.path.join(DATA_DIR, 'rhea_all2master_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)

## REF_RHEA2LABEL

Previous version used the names of the reactions as labels. However, the names are not always unique, and some of them may be confusing or hard to understand. Therefore, I used the equations directly as labels.

In [43]:
rhea2label = dict()

for idx in df.index:
  one_row = df.loc[idx, :]
  one_mrhea = all2rhea_master[one_row['Reaction identifier']]
  equations = one_row['Equation'].split(';')
  rhea2label[one_mrhea] = equations

compress_pickle.dump(rhea2label, os.path.join(DATA_DIR, 'rhea2label_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)

## REF_RHEA2CHEBI + REF_RHEA2FORMULA

1. REF_RHEA2CHEBI was called 'mrhea2chebi_prime.lzma' in AMAS, because it used to map secondary chebi ids to prime chebi ids. However, after testing, I found that the rhea reference file already uses the prime chebi ids (and there are only two exceptions), therefore, I just used the Rhea data directly here.
  
2. REF_RHEA2FORMULA is a dictionary of rhea to chebi formulas. This is not used in AMAS, but is used to create the reference matrix REF_MAT.


In [ ]:
# Open the chebi formula file 
# created by `notebooks/create_chebi_reference_cleaned.ipynb`
with open(os.path.join(DATA_DIR, 'chebi_shortened_formula_2jan2025.lzma'), 'rb') as f:
  chebi2sformula = compress_pickle.load(f)

In [53]:
mrhea2chebi = dict()
mrhea2sformula = dict()
for idx in df.index:
  one_row = df.loc[idx, :]
  one_mrhea = all2rhea_master[one_row['Reaction identifier']]
  chebi_ids = one_row['ChEBI identifier'].split(';')
  sforms = list(set([chebi2sformula[val] for val in chebi_ids if val in chebi2sformula.keys()]))
  if len(sforms)>0:
    mrhea2chebi[one_mrhea] = chebi_ids
    mrhea2sformula[one_mrhea] = sforms

In [56]:
compress_pickle.dump(mrhea2chebi, os.path.join(DATA_DIR, 'mrhea2chebi_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)
compress_pickle.dump(mrhea2sformula, os.path.join(DATA_DIR, 'mrhea2sformula_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)

## REF-MAT (or REF-DAT) 

### REF_MAT
This is the reference matrix of the reactions, where the rows are the reactions and the columns are the chebi formulas, and the values are 1 if the formula exists in the reaction, and 0 otherwise.

### REF_DAT
This is the reduced size version of REF_MAT, the file is called 'data2ref_mat.lzma' in AMAS. 

It is a tuple of three elements:
1. list of chebi formulas
2. list of rhea terms
3. list of indices of non-zero values in the reference matrix (to represent the existance of a formula in a reaction)


In [59]:
# collect all formulas (columns)
formulas = []
for one_k in mrhea2sformula.keys():
  one_list_formula = mrhea2sformula[one_k]
  formulas.append(one_list_formula)
all_formulas = list(set(itertools.chain(*formulas)))

# set index as the list of Rhea terms
ref_mat = pd.DataFrame(0, index=list(mrhea2sformula.keys()), columns=all_formulas)

for one_k in mrhea2sformula.keys():
  one_list_formula = mrhea2sformula[one_k]
  ref_mat.loc[one_k, one_list_formula] = 1

print(ref_mat.shape)

(17131, 4286)


In [70]:
# ref_dat is a decomposed version of ref_mat, to reduce size
# do it by columns (smaller number than rhea reactions)
nonzero_vals = []
for cidx in range(len(ref_mat.columns)):
  one_col = ref_mat.iloc[:, cidx]
  nonzero_vals.append((cidx, list(one_col.to_numpy().nonzero()[0])))
ref_dat = (all_formulas, list(mrhea2sformula.keys()), nonzero_vals)

print(len(ref_dat[0]))
print(len(ref_dat[1]))
print(len(ref_dat[2]))

4286
17131
4286


In [72]:
compress_pickle.dump(ref_dat, os.path.join(DATA_DIR, 'data2ref_mat_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)

## REF_RHEA2ECKEGG, REF_EC2RHEA, REF_KEGG2RHEA
These are the mappings of the reactions (master direction) to the EC and KEGG terms.

In [103]:
mrhea2eckegg = {} 
ec2mrhea = {}
kegg2mrhea = {}
for _, row in df.iterrows():
    mrhea = all2rhea_master[row["Reaction identifier"]]
    ec_number = row["EC number"]
    # Extract EC number after 'EC:'
    ec_extracted = ec_number.split("EC:")[-1].strip() if pd.notna(ec_number) and "EC:" in ec_number else None
    
    kegg_ref = row["Cross-reference (KEGG)"]
    # Extract KEGG ID after 'KEGG:'
    kegg_extracted = kegg_ref.split("KEGG:")[-1].strip() if pd.notna(kegg_ref) and "KEGG:" in kegg_ref else None
    
    # Store only non-None values in the tuple
    values = list(filter(None, (ec_number, ec_extracted, kegg_ref, kegg_extracted)))
    if values: 
        mrhea2eckegg[mrhea] = values
    
    if ec_number not in ec2mrhea:
        ec2mrhea[ec_number] = [] 
    ec2mrhea[ec_number].append(mrhea)

    if kegg_ref not in kegg2mrhea:
        kegg2mrhea[kegg_ref] = []
    kegg2mrhea[kegg_ref].append(mrhea)

print(len(mrhea2eckegg))
print(len(ec2mrhea))
print(len(kegg2mrhea))

8521
6042
6820


In [106]:
compress_pickle.dump(mrhea2eckegg, os.path.join(DATA_DIR, 'mrhea2eckegg_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)
compress_pickle.dump(ec2mrhea, os.path.join(DATA_DIR, 'ec2mrhea_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)
compress_pickle.dump(kegg2mrhea, os.path.join(DATA_DIR, 'kegg2mrhea_jan2025.lzma'),
                     compression="lzma", set_default_extension=False)